# Case Study: Maaz et al, 2025

This is the code and description for the case study in the paper. It is a re-analysis of the paper "Cost-effectiveness of drone-delivered automated external defibrillators for cardiac arrest" by Maaz et al, 2025 (Resuscitation). The model in their paper is essentially a Markov reward process with fixed $P$ and $r$, with $\pi$ given by an XGBoost model. We obtained the data from the authors, and instead used a neural network for the sake of this analysis. Their Markov process has 20 states: discharged with a Rankin scale 0-6, first-year post-discharge with mRS 0-5, later-year post-discharge with mRS 0-5, plus death. The only possible transitions are, for a given mRS: discharge to first-year or death, first-year to later-year or death, and later-year to itself or death. Every patient starts at one of the discharge states (i.e., 7 possible starting states), with the rest of them assigned 0 probability mass. Which mRS state the patient begins is predicted from the patient's features, characteristics of the OHCA, as well as the time to defibrillation. So, these probabilities are set equal to the output of a multiclass classifier.

Due to data privacy issues, we can only share the *model* trained on the data, not the data itself. However, this is enough for our analysis, as all we need is the Markov process as well as the model.

In [13]:
# to be able to import package
import sys; sys.path.append("../")

import gurobipy as gp
from gurobipy import GRB
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Sequential
from markovml.utils.models_ext import SequentialClassifier
from markovml.markovml import MarkovReward


In [14]:
# build the MRP
mrp = MarkovReward(n_states=20, n_features=5, discount_factor=0.97)

In [15]:
# data from the paper

transition_matrix = [
    # dis_0 first_0 later_0 dis_1 first_1 later_1 dis_2 first_2 later_2 dis_3 first_3 later_3 dis_4 first_4 later_4 dis_5 first_5 later_5 dis_6 death
    [0,     1,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0],     # dis_0
    [0,     0,      0.987,  0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0.013], # first_0
    [0,     0,      0.955,  0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0.045], # later_0
    [0,     0,      0,      0,    1,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0],     # dis_1
    [0,     0,      0,      0,    0,      0.939,  0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0.061], # first_1
    [0,     0,      0,      0,    0,      0.977,  0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0.023], # later_1
    [0,     0,      0,      0,    0,      0,      0,    1,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0],     # dis_2
    [0,     0,      0,      0,    0,      0,      0,    0,      0.944,  0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0.056], # first_2
    [0,     0,      0,      0,    0,      0,      0,    0,      0.951,  0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0.049], # later_2
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    1,      0,      0,    0,      0,      0,    0,      0,      0,    0],     # dis_3
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0.851,  0,    0,      0,      0,    0,      0,      0,    0.149], # first_3
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0.953,  0,    0,      0,      0,    0,      0,      0,    0.047], # later_3
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    1,      0,      0,    0,      0,      0,    0],     # dis_4
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0.748,  0,    0,      0,      0,    0.252], # first_4
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0.889,  0,    0,      0,      0,    0.111], # later_4
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    1,      0,      0,    0],     # dis_5
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0.402,  0,    0.598], # first_5
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0.904,  0,    0.096], # later_5
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    1],     # dis_6
    [0,     0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    0,      0,      0,    1],     # death
]

utils = [
    0,     # dis_0
    1,     # first_0
    1,     # later_0
    0,     # dis_1
    0.84,  # first_1
    0.84,  # later_1
    0,     # dis_2
    0.78,  # first_2
    0.78,  # later_2
    0,     # dis_3
    0.71,  # first_3
    0.71,  # later_3
    0,     # dis_4
    0.44,  # first_4
    0.44,  # later_4
    0,     # dis_5
    0.18,  # first_5
    0.18,  # later_5
    0,     # dis_6
    0      # death
]

costs = [
    54494.26794,   # dis_0
    3687.307578,   # first_0
    3830.155927,   # later_0
    81686.92181,   # dis_1
    7822.161121,   # first_1
    5120.287055,   # later_1
    128987.9481,   # dis_2
    14611.31963,   # first_2
    8523.621354,   # later_2
    226696.1769,   # dis_3
    30197.0023,    # first_3
    22187.18021,   # later_3
    281026.9736,   # dis_4
    65906.93811,   # first_4
    55509.91759,   # later_4
    225442.8067,   # dis_5
    74957.59677,   # first_5
    52085.62883,   # later_5
    22833.09511,   # dis_6
    0              # death
]

rewards = 150000 * np.array(utils) - np.array(costs)

In [16]:
# set the MRP parameters
mrp.set_P(transition_matrix)
mrp.set_r(rewards)


In [17]:
# load the ML model
clf = torch.load("pred_model.pth", weights_only=False)
clf = SequentialClassifier(clf[:-1]) # sequential classifier expects final layer to be linear, and adds the softmax

# sanity check
print(clf)

SequentialClassifier(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=7, bias=True)
  (3): Softmax(dim=1)
)


In [18]:
# add ml model
mrp.add_ml_model(clf)


In [19]:
# link pi to ml models
mrp.set_pi([
    mrp.ml_outputs[0][0],     # dis_0
    0,     # first_0
    0,     # later_0
    mrp.ml_outputs[0][1],     # dis_1
    0,  # first_1
    0,  # later_1
    mrp.ml_outputs[0][2],     # dis_2
    0,  # first_2
    0,  # later_2
    mrp.ml_outputs[0][3],     # dis_3
    0,  # first_3
    0,  # later_3
    mrp.ml_outputs[0][4],     # dis_4
    0,  # first_4
    0,  # later_4
    mrp.ml_outputs[0][5],     # dis_5
    0,  # first_5
    0,  # later_5
    mrp.ml_outputs[0][6],     # dis_6
    0      # death
])

In [20]:
# set constraints
# features are, in order: time to defibrillation (seconds), age, sex (binary, 1=male), witnessed (binary), shockable (binary)
mrp.add_feature_constraint(mrp.features[0] >= 200) # 3 minutes
mrp.add_feature_constraint(mrp.features[0] <= 360) # 6 minutes

mrp.add_feature_constraint(mrp.features[1] >= 50) # 60 years
mrp.add_feature_constraint(mrp.features[1] <= 75) # 100 years

mrp.features[2].VType = GRB.BINARY
mrp.features[3].VType = GRB.BINARY
mrp.features[4].VType = GRB.BINARY

mrp.add_feature_constraint(mrp.features[2] == 1)
mrp.add_feature_constraint(mrp.features[3] == 1)
mrp.add_feature_constraint(mrp.features[4] == 1)



In [12]:
# optimize the MRP
mrp.optimize(verbose=True)

Set parameter OutputFlag to value 1
Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.3.0 23D2057)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0

Optimize a model with 22 rows, 35 columns and 118 nonzeros
Model fingerprint: 0x93784a2c
Model has 8 simple general constraints
  8 MAX
Model has 7 general nonlinear constraints (63 nonlinear terms)
Variable types: 32 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-03, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 4e+02]
Presolve model has 7 nlconstr
Added 15 variables to disaggregate expressions.
         in nonlinear terms.
         Consider bounding these variables or reformulating the model.


Solving non-convex MINLP

Variable types: 64 continuous, 19 integer (19 binary)

Root relaxation: objective 0.000000e+